In [2]:
from ultralytics import YOLO

In [3]:
# Load a pretrained YOLO model (recommended for training)
model = YOLO("yolov8n.pt")

In [ ]:
# Train the model using the 'coco128.yaml' dataset for 3 epochs
#results = model.train(data='coco128.yaml', epochs=5)

# Evaluate the model's performance on the validation set
#results = model.val()

In [ ]:
# Export the model to ONNX format
success = model.export(format='pb')

In [7]:
import os

path = "datasets/mango/images/test/raw"

# List all files in the directory
all_files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
for file in all_files:
    f = os.path.join(path, file)
    results = model.predict(f, show=True)



image 1/1 /Users/jacopouggeri/workspace/datathon/datasets/mango/images/test/raw/Raw_Mango_0_6461.jpg: 480x640 1 person, 1 banana, 1 apple, 96.2ms
Speed: 2.2ms preprocess, 96.2ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/jacopouggeri/workspace/datathon/datasets/mango/images/test/raw/Raw_Mango_0_7964.jpg: 480x640 1 banana, 52.5ms
Speed: 2.1ms preprocess, 52.5ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/jacopouggeri/workspace/datathon/datasets/mango/images/test/raw/Raw_Mango_0_3653.jpg: 480x640 1 orange, 54.5ms
Speed: 1.4ms preprocess, 54.5ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/jacopouggeri/workspace/datathon/datasets/mango/images/test/raw/Raw_Mango_0_3121.jpg: 480x640 2 apples, 48.1ms
Speed: 1.0ms preprocess, 48.1ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/jacopouggeri/workspace/datathon/datasets/mango/images/test/raw/

KeyboardInterrupt: 

: 

In [ ]:
import os
import cv2
import numpy as np
from helper import resize_and_pad

folder_type = "train/ripe" # test/train raw/ripe

input_folder = 'datasets/mango/images/' + folder_type
output_folder = 'datasets/mango_cut/images/' + folder_type
confidence_threshold = 0.6  # Adjust as needed

image_files = [f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]

# Create the folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Delete all files in the output folder
for filename in os.listdir(output_folder):
    file_path = os.path.join(output_folder, filename)
    if os.path.isfile(file_path):  # ensure it's a file, not a directory or other type
        os.remove(file_path)

for image_file in image_files:
    # Load the image using cv2
    img_path = os.path.join(input_folder, image_file)
    img = cv2.imread(img_path)
    
    # Generate predictions for the image
    results = model.predict(source=img)
    
    # Check if any detection was made
    if len(results[0].boxes.xyxy) == 0:
        print(f"Skipping {image_file} due to no detection.")
        continue

    # Get the index of the bounding box with the highest confidence
    max_conf_index = np.argmax(results[0].boxes.conf.cpu().numpy())
    
    # Check if highest confidence is above threshold
    if results[0].boxes.conf[max_conf_index] < confidence_threshold:
        print(f"Skipping {image_file} due to low confidence.")
        continue
    
    # Extract and save the object with highest confidence
    box = results[0].boxes.xyxy[max_conf_index].cpu().numpy().astype(int)
    x1, y1, x2, y2 = box
    cropped_img = img[y1:y2, x1:x2]

    # Resize and pad
    final_img = resize_and_pad(cropped_img)

    # Save the cropped image
    output_path = os.path.join(output_folder, f"crop_{image_file}")
    cv2.imwrite(output_path, final_img)

In [4]:
import os
import cv2

path = "datasets/mango/images/test/ripe"

# List all files in the directory
all_files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]

for file in all_files:
    f = os.path.join(path, file)
    
    # Predict using the model
    results = model.predict(f)
    
    # Read the image
    img = cv2.imread(f)
    
    # Draw bounding boxes on the image
    for i, box in enumerate(results[0].boxes.xyxy):
        # Extract the bounding box coordinates
        x1, y1, x2, y2 = map(int, box)
        
        # Draw the rectangle on the image
        cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)  # Green color for the rectangle
        
        # Optionally: Put the label (class) on the top of the bounding box
        # label = str(results.classes[i])
        # cv2.putText(img, label, (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    
    # Display the image with boxes
    cv2.imshow('Detected Mangoes', img)
    cv2.waitKey(0)  # Waits for a key press to move to the next image

cv2.destroyAllWindows()



image 1/1 /Users/jacopouggeri/workspace/datathon/datasets/mango/images/test/ripe/Ripe_Mango_0_7546.jpg: 480x640 1 person, 2 apples, 59.3ms
Speed: 1.3ms preprocess, 59.3ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
2023-10-22 00:31:17.331 Python[93421:4647387] WARNING: Secure coding is not enabled for restorable state! Enable secure coding by implementing NSApplicationDelegate.applicationSupportsSecureRestorableState: and returning YES.

image 1/1 /Users/jacopouggeri/workspace/datathon/datasets/mango/images/test/ripe/Ripe_Mango_0_7156.jpg: 480x640 1 sports ball, 1 apple, 47.6ms
Speed: 1.6ms preprocess, 47.6ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/jacopouggeri/workspace/datathon/datasets/mango/images/test/ripe/Ripe_Mango_0_2990.jpg: 480x640 3 oranges, 51.1ms
Speed: 1.6ms preprocess, 51.1ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)

image 1/1 /Users/jacopouggeri/workspace/datathon/datasets/mango/ima

KeyboardInterrupt: 

: 